In [ ]:
import pandas as pd

# Ruta al archivo zip
ruta = "/documents/input/ecobici-v2/datos_modificadosV2.csv"

# Leer archivo CSV comprimido en zip
datos = pd.read_csv(ruta)

# Verificar que se haya unido bien
print(datos.head())

df = datos
# Convertir las columnas de fechas a tipo datetime
df['FechaHora_Retiro'] = pd.to_datetime(df['FechaHora_Retiro'])
df['FechaHora_Arribo'] = pd.to_datetime(df['FechaHora_Arribo'])

def limpiar_estacion(estacion):
    estacion = str(estacion)
    if '.' in estacion:
        estacion = str(int(float(estacion)))
    if '-' in estacion:
        if estacion.startswith('CE-'):
            estacion = estacion.split('-')[1]  
        else:
            estacion = estacion.split('-')[0]  
    return estacion.zfill(3)  

df['Ciclo_Estacion_Retiro'] = df['Ciclo_Estacion_Retiro'].apply(limpiar_estacion)
df['Ciclo_Estacion_Arribo'] = df['Ciclo_Estacion_Arribo'].apply(limpiar_estacion)

In [ ]:
prom_duracion_hora = df.groupby(df['FechaHora_Retiro'].dt.hour)['Duracion_Minutos'].mean()
prom_duracion_hora.plot(kind='line', marker='o', title='Duración Promedio por Hora del Día')
plt.ylabel('Minutos')
plt.xlabel('Hora')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Filtrar los datos desde 2018 en adelante
df_filtrado = df[df['FechaHora_Retiro'].dt.year >= 2018]

# Agrupar por año
viajes_por_anio = df_filtrado.groupby(df_filtrado['FechaHora_Retiro'].dt.year).size()

# Crear figura y eje
fig, ax = plt.subplots()

# Graficar
viajes_por_anio.plot(kind='bar', color='skyblue', edgecolor='black', ax=ax)

# Títulos y etiquetas
ax.set_title('Cantidad Total de Viajes por Año (2018–2024)', fontsize=10)
ax.set_xlabel('Año', fontsize=10)
ax.set_ylabel('Cantidad de viajes', fontsize=10)
ax.set_xticklabels(viajes_por_anio.index, rotation=0)

# Formato del eje Y en millones
formatter = FuncFormatter(lambda x, _: f'{x/1e6:.1f}M')
ax.yaxis.set_major_formatter(formatter)

# Quitar fondo y rejilla
ax.set_facecolor('none')
fig.patch.set_facecolor('none')  

plt.tight_layout()
plt.show()

In [ ]:
# Filtrar solo desde 2018 en adelante
df_filtrado = df[df['FechaHora_Retiro'].dt.year >= 2018]

# Agrupar por año y mes, contar viajes
viajes_anuales = (
    df_filtrado
    .groupby([df_filtrado['FechaHora_Retiro'].dt.year, df_filtrado['FechaHora_Retiro'].dt.month])
    .size()
    .unstack(level=0)
)

# Crear el gráfico
fig, ax = plt.subplots()
viajes_anuales.plot(ax=ax, marker='', title='Viajes por Mes (2018–2024)', xlabel='Mes', ylabel='Cantidad de viajes')

# Personalizar fuentes
ax.set_title('Viajes por Mes (2018–2024)', fontsize=10)
ax.set_xlabel('Mes', fontsize=10)
ax.set_ylabel('Cantidad de viajes', fontsize=10)

# Configurar ticks y formato del eje Y
plt.xticks(range(1, 13))
ax.grid(False) 

formatter = FuncFormatter(lambda x, _: f'{x/1e6:.1f}M')
ax.yaxis.set_major_formatter(formatter)

plt.legend(title='Año')
plt.tight_layout()
plt.show()

In [ ]:
# Agrupar sin crear nuevas columnas en el df
viajes_hora_dia = (
    df.groupby([df['FechaHora_Retiro'].dt.hour, df['FechaHora_Retiro'].dt.dayofweek])
    .size()
    .unstack()
)

# Renombrar columnas a días
dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
viajes_hora_dia.columns = dias

# Graficar
ax = viajes_hora_dia.plot(figsize=(10, 6), marker='')
plt.title('Viajes por Hora según el Día de la Semana', fontsize=10)
plt.xlabel('Hora del Día', fontsize=10)
plt.ylabel('Cantidad de viajes', fontsize=10)
plt.xticks(range(0, 24))
plt.grid(False)
plt.legend(title='Día de la Semana')
plt.tight_layout()

# Formatear eje Y en millones
formatter = FuncFormatter(lambda x, _: f'{x/1e6:.1f}M')
ax.yaxis.set_major_formatter(formatter)

plt.show()

In [ ]:
# Años a incluir (sin 2017)
anios = sorted(df['FechaHora_Retiro'].dt.year.unique())
anios = [anio for anio in anios if anio >= 2018]

# Crear subplots (ajusta según cantidad de años)
n = len(anios)
filas = 2
columnas = 4
fig, axs = plt.subplots(nrows=filas, ncols=columnas, figsize=(12, 8), sharex=True, sharey=True)
axs = axs.flatten()

dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

for i, anio in enumerate(anios):
    df_anio = df[df['FechaHora_Retiro'].dt.year == anio]
    
    viajes_hora_dia = (
        df_anio.groupby([df_anio['FechaHora_Retiro'].dt.hour, df_anio['FechaHora_Retiro'].dt.dayofweek])
        .size()
        .unstack()
    )
    viajes_hora_dia.columns = dias

    ax = axs[i]
    viajes_hora_dia.plot(ax=ax, legend=False)
    ax.set_title(f'Año {anio}', fontsize=10)
    ax.set_xlabel('Hora', fontsize=10)
    ax.set_ylabel('Viajes', fontsize=10)
    ax.xaxis.set_ticks(range(0, 24, 3))
    
    formatter = FuncFormatter(lambda x, _: f'{x/1e3:.1f}K') 
    ax.yaxis.set_major_formatter(formatter)
    ax.grid(False)

# Eliminar subplots vacíos si sobran
for j in range(i + 1, len(axs)):
    fig.delaxes(axs[j])

# Agregar leyenda general
handles, labels = axs[0].get_legend_handles_labels()

fig.legend(handles, labels, title='Día de la Semana', loc='lower center', ncol=7, bbox_to_anchor=(0.5, -0.05))
plt.tight_layout(rect=[0, 0.05, 1, 0.95]) 
plt.suptitle('Distribución Horaria de Viajes por Año (2018–2024)', fontsize=12) 


In [ ]:
# Diccionario para los días de la semana
dias_semana = {0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves', 4: 'Viernes', 5: 'Sábado', 6: 'Domingo'}

# Pivot table
pivot = df.pivot_table(index=df['FechaHora_Retiro'].dt.dayofweek,
                       columns=df['FechaHora_Retiro'].dt.hour,
                       values='Bici', aggfunc='count')

# Reemplazar los índices por los nombres de los días
pivot.index = pivot.index.map(dias_semana)

# Graficar mapa de calor
sns.heatmap(pivot, cmap="YlGnBu")
plt.title('Mapa de Calor: Viajes por Día de la Semana y Hora', fontsize=10)
plt.ylabel('Día de la Semana', fontsize=10)
plt.xlabel('Hora del Día', fontsize=10)

# Ajustar ticks con fontsize 10
plt.xticks(fontsize=10)
plt.yticks(fontsize=10, rotation=0)
plt.show() 

In [ ]:
# Contar los géneros
genero_counts = df['Genero_Usuario'].value_counts()
labels = genero_counts.index
sizes = genero_counts.values

# Paleta original
colors = ['skyblue', '#ff9999', '#99ff99', '#ffcc99']

# Crear gráfico
fig, ax = plt.subplots(figsize=(5, 5))
wedges, texts, autotexts = ax.pie(
    sizes,
    autopct='%1.1f%%',
    startangle=100,
    colors=colors,
    textprops={'fontsize': 14, 'color': 'black'},
    pctdistance=0.7
)

# Leyenda
ax.legend(wedges, labels, title='Género', loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, title_fontsize=11)

# Título más cerca y centrado
fig.suptitle('Distribución de Género', fontsize=12, y=0.92)

# Ajustes finales
ax.axis('equal')
plt.tight_layout()
plt.show()


In [ ]:
# Gráfico de distribución de edades
plt.figure(figsize=(10, 6))
n, bins, patches = plt.hist(df['Edad_Usuario'], bins=20, color='skyblue', edgecolor='black')

# Formatear el eje Y en millones
formatter = FuncFormatter(lambda x, _: f'{x/1e6:.1f}M')  # Divide por 1e6 y muestra con un decimal
plt.gca().yaxis.set_major_formatter(formatter)

plt.title('Distribución de Edades de los Usuarios', fontsize=12)
plt.xlabel('Edad', fontsize=10)
plt.ylabel('Frecuencia (en millones)', fontsize=10)
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
# Agrupar edades en rangos sin crear columna
edad_bins = [16, 25, 35, 45, 55, 80]  # 80 sigue siendo el límite superior para el grupo 56+
edad_labels = ['16-25', '26-35', '36-45', '46-55', '56+']  # Cambié el grupo superior
edad_grupos = pd.cut(df['Edad_Usuario'], bins=edad_bins, labels=edad_labels)

# Contar usuarios por grupo de edad
grupo_counts = edad_grupos.value_counts().sort_index()

# Gráfico de torta
fig, ax = plt.subplots(figsize=(6, 6))
colors = ['skyblue', '#ff9999', '#99ff99', '#ffcc99', '#ff6666']  # Mantuve los colores

wedges, texts, autotexts = ax.pie(
    grupo_counts.values,
    labels=None,
    autopct='%1.1f%%',
    startangle=100,
    colors=colors,
    textprops={'fontsize': 14, 'color': 'black'},
    pctdistance=0.80
)

# Leyenda
ax.legend(wedges, grupo_counts.index, title='Grupo de Edad', loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, title_fontsize=11)

# Título centrado y más cerca
fig.suptitle('Distribución de Usuarios por Grupo de Edad', fontsize=12, y=0.92)

ax.axis('equal')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

# Definir grupos de edad
edad_bins = [16, 25, 35, 45, 55, 80]
edad_labels = ['16-25', '26-35', '36-45', '46-55', '56+']

# Filtrar viajes donde retiro ≠ arribo
mask = df['Ciclo_Estacion_Retiro'] != df['Ciclo_Estacion_Arribo']

# Crear las series necesarias para el MultiIndex sin modificar el DataFrame original
grupo_edad = pd.cut(df.loc[mask, 'Edad_Usuario'], bins=edad_bins, labels=edad_labels)
rutas = list(zip(df.loc[mask, 'Ciclo_Estacion_Retiro'], df.loc[mask, 'Ciclo_Estacion_Arribo']))

# Crear una Series con MultiIndex y contar frecuencia de rutas
conteo_series = pd.Series(1, index=pd.MultiIndex.from_arrays([grupo_edad, rutas], names=['Edad_Grupo', 'Ruta']))
rutas_comunes = conteo_series.groupby(level=[0, 1], observed=True).sum().reset_index(name='Frecuencia')

# Obtener las 5 rutas más frecuentes por grupo de edad
top_5_por_grupo = (
    rutas_comunes
    .sort_values(['Edad_Grupo', 'Frecuencia'], ascending=[True, False])
    .groupby('Edad_Grupo', group_keys=False)
    .head(5)
    .reset_index(drop=True)
)

print(top_5_por_grupo)


In [ ]:
# Crear tabla final con columnas = grupos de edad y filas = top 1-5
tabla_final = pd.DataFrame(index=range(1, 6))

for grupo in edad_labels:
    rutas_top = top_5_por_grupo[top_5_por_grupo['Edad_Grupo'] == grupo]['Ruta'].tolist()
    # Rellenar con None si hay menos de 5 rutas
    tabla_final[grupo] = rutas_top + [None] * (5 - len(rutas_top))

tabla_final.index.name = 'Top'
tabla_final.columns.name = 'Grupo de edad'

print(tabla_final)

In [ ]:
duracion_prom = df.groupby(pd.cut(df['Edad_Usuario'], bins=edad_bins, labels=edad_labels))['Duracion_Minutos'].mean()
print(duracion_prom.head())

In [ ]:
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_horas_grafico, x='Hora', y='Frecuencia', hue='Grupo de edad', marker='o')
plt.title('Distribución horaria del uso de Ecobici por grupo de edad')
plt.xlabel('Hora del día')
plt.ylabel('Número de viajes')
plt.xticks(range(0, 24))
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
# Definir los grupos de edad al vuelo
bins = [15, 25, 35, 45, 55, 150]
labels = ['16-25', '26-35', '36-45', '46-55', '56+']

# Agrupar por hora, grupo de edad y ruta
tabla = (
    df[df['Ciclo_Estacion_Retiro'] != df['Ciclo_Estacion_Arribo']]
    .assign(
        Hora=df['FechaHora_Retiro'].dt.hour,
        Edad_Grupo=pd.cut(df['Edad_Usuario'], bins=bins, labels=labels)
    )
    .groupby(['Hora', 'Edad_Grupo', 'Ciclo_Estacion_Retiro', 'Ciclo_Estacion_Arribo'])
    .size()
    .reset_index(name='Frecuencia')
)

# Obtener la ruta más frecuente por grupo de edad y hora
ruta_mas_comun = (
    tabla.sort_values(['Hora', 'Edad_Grupo', 'Frecuencia'], ascending=[True, True, False])
    .groupby(['Hora', 'Edad_Grupo'], as_index=False)
    .first()
)

print(ruta_mas_comun)


In [ ]:
ruta_mas_comun['Ruta'] = ruta_mas_comun.apply(
    lambda x: f"{x['Ciclo_Estacion_Retiro']}→{x['Ciclo_Estacion_Arribo']}", axis=1
)

heatmap_data = ruta_mas_comun.pivot(index='Edad_Grupo', columns='Hora', values='Frecuencia')

plt.figure(figsize=(18, 6))
sns.heatmap(heatmap_data, annot=False, cmap='YlGnBu', cbar_kws={'label': 'Frecuencia'})

for _, row in ruta_mas_comun.iterrows():
    edad = row['Edad_Grupo']
    hora = row['Hora']
    ruta = row['Ruta']
    y = heatmap_data.index.get_loc(edad) + 0.5
    x = hora + 0.5
    valor_normalizado = heatmap_data.loc[edad, hora] / heatmap_data.values.max()
    color_texto = 'white' if valor_normalizado > 0.5 else 'black'
    plt.text(x, y, ruta, ha='center', va='center', fontsize=10.5, color=color_texto, rotation=45)


plt.title('Ruta más frecuente por hora y grupo de edad', fontsize=14)
plt.ylabel('Grupo de edad', fontsize=12)
plt.xlabel('Hora del día', fontsize=12)

plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Top 10 estaciones de retiro y arribo (orden ascendente para que se vea bien en horizontal)
top_retiro = df['Ciclo_Estacion_Retiro'].value_counts().head(10).sort_values(ascending=True)
top_arribo = df['Ciclo_Estacion_Arribo'].value_counts().head(10).sort_values(ascending=True)

# Crear figura con dos subplots horizontales
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico de estaciones de retiro
top_retiro.plot(kind='barh', color='skyblue', ax=axes[0])
axes[0].set_title('Top 10 Estaciones de Retiro')
axes[0].set_xlabel('Cantidad de viajes (miles)')
axes[0].set_ylabel('Estación de Retiro')
axes[0].set_xticks(axes[0].get_xticks())
axes[0].set_xticklabels([f'{int(x/1000)}k' for x in axes[0].get_xticks()])

# Gráfico de estaciones de arribo
top_arribo.plot(kind='barh', color='#ff9999', ax=axes[1])
axes[1].set_title('Top 10 Estaciones de Arribo')
axes[1].set_xlabel('Cantidad de viajes (miles)')
axes[1].set_ylabel('Estación de Arribo')
axes[1].set_xticks(axes[1].get_xticks())
axes[1].set_xticklabels([f'{int(x/1000)}k' for x in axes[1].get_xticks()])

plt.tight_layout()
plt.show()

In [ ]:
dobles_retiro = df[df['Ciclo_Estacion_Retiro'].astype(str).str.contains('-')]['Ciclo_Estacion_Retiro'].unique()
dobles_arribo = df[df['Ciclo_Estacion_Arribo'].astype(str).str.contains('-')]['Ciclo_Estacion_Arribo'].unique()

print("Estaciones dobles en retiro:")
print(dobles_retiro)

print("\nEstaciones dobles en arribo:")
print(dobles_arribo)

In [ ]:
# HEATMAP PARA RETIROS
heatmap_retiro = (
    df.assign(Año=df['FechaHora_Retiro'].dt.year)
    .query("Año >= 2018")
    .groupby(['Ciclo_Estacion_Retiro', 'Año'])
    .size()
    .reset_index(name='Cantidad')
    .pivot(index='Ciclo_Estacion_Retiro', columns='Año', values='Cantidad')
    .fillna(0) / 1000  # valores en miles
)

top_estaciones_retiro = heatmap_retiro.sum(axis=1).sort_values(ascending=False).head(10).index
tabla_top_retiro = heatmap_retiro.loc[top_estaciones_retiro]

# HEATMAP PARA ARRIBOS
heatmap_arribo = (
    df.assign(Año=df['FechaHora_Arribo'].dt.year)
    .query("Año >= 2018")
    .groupby(['Ciclo_Estacion_Arribo', 'Año'])
    .size()
    .reset_index(name='Cantidad')
    .pivot(index='Ciclo_Estacion_Arribo', columns='Año', values='Cantidad')
    .fillna(0) / 1000  # valores en miles
)

top_estaciones_arribo = heatmap_arribo.sum(axis=1).sort_values(ascending=False).head(10).index
tabla_top_arribo = heatmap_arribo.loc[top_estaciones_arribo]

# 🔵 Heatmap Retiro
tabla_top_retiro.index = tabla_top_retiro.index.astype(str)
plt.figure(figsize=(8, 4))
sns.heatmap(tabla_top_retiro, annot=True, fmt=".0f", cmap="Blues")
plt.title("Top estaciones de Retiro por Año (en miles)")
plt.xlabel("Año")
plt.ylabel("Estación de Retiro")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


# 🔴 Heatmap Arribo
tabla_top_arribo.index = tabla_top_arribo.index.astype(str)
plt.figure(figsize=(8, 4))
sns.heatmap(tabla_top_arribo, annot=True, fmt=".0f", cmap="Reds")
plt.title("Top estaciones de Arribo por Año (en miles)")
plt.xlabel("Año")
plt.ylabel("Estación de Arribo")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd

# Ruta al archivo zip
ruta3 = "/kaggle/input/estaciones-ecobici/ecobici_estaciones_con_numero.csv"

# Leer archivo CSV comprimido en zip
datos3 = pd.read_csv(ruta3)

print(datos3.head())

In [ ]:
# Crear tabla con estaciones top y su ubicación
tabla_estaciones = retiro_coords[['Número', 'Latitud', 'Longitud']]

# Mostrar tabla
print("Top 20 estaciones de retiro con su ubicación:")
display(tabla_estaciones) 


In [ ]:
from folium.plugins import HeatMap
import folium

# Diccionario para guardar los mapas
heatmaps_por_año = {}

# Filtramos solo años que existen en los datos
años_disponibles = sorted(df['FechaHora_Retiro'].dt.year.unique())
años_disponibles = [año for año in años_disponibles if año >= 2018]  # solo de 2018 en adelante

for año in años_disponibles:
    # Filtrar viajes de ese año
    df_año = df[df['FechaHora_Retiro'].dt.year == año]
    
    # Estaciones utilizadas ese año
    estaciones_retiro = df_año['Ciclo_Estacion_Retiro'].dropna().unique()
    estaciones_retiro = [str(e).zfill(3) for e in estaciones_retiro]
    
    # Coordenadas de esas estaciones
    estaciones_activas = datos3[datos3['Número'].isin(estaciones_retiro)]

    # Centro del mapa
    lat_centro = estaciones_activas['Latitud'].mean()
    lon_centro = estaciones_activas['Longitud'].mean()

    # Crear mapa base
    mapa = folium.Map(
        location=[lat_centro, lon_centro],
        zoom_start=13,
        tiles="https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png",
        attr='© OpenStreetMap contributors © CARTO'
    )

    folium.GeoJson(
        colonias_geojson,
        name='Colonias CDMX',
        style_function=lambda feature: {
            'fillColor': 'green',
            'color': 'lightgrey',
            'weight': 0.5,
            'fillOpacity': 0.1
        }
    ).add_to(mapa)

    # Crear lista de puntos para el HeatMap
    puntos = estaciones_activas[['Latitud', 'Longitud']].values.tolist()

    # Añadir HeatMap
    HeatMap(puntos, radius=15, blur=10, min_opacity=0.4).add_to(mapa)

    # Guardar mapa en el diccionario
    heatmaps_por_año[año] = mapa

# Ejemplo para visualizar el mapa de 2018
heatmaps_por_año[2018]


In [ ]:
# Ejemplo para visualizar el mapa de 2022
heatmaps_por_año[2020]

In [ ]:
import folium
import json

# Cargar archivo geojson de colonias
with open('/kaggle/input/cdmx-geojsonv2/coloniascdmx.geojson', 'r', encoding='utf-8') as f:
    colonias_geojson = json.load(f)


mapa = folium.Map(
    location=[19.4326, -99.1332],
    zoom_start=12,
    tiles="Stamen Toner",  # Mapa en blanco y negro
    attr='© OpenStreetMap contributors © Stamen Design'
)

folium.GeoJson(
    colonias_geojson,
    name='Colonias CDMX',
    style_function=lambda feature: {
        'fillColor': 'grey',
        'color': 'darkgrey',
        'weight': 1,
        'fillOpacity': 0.2
    }
).add_to(mapa)



for _, fila in datos3.iterrows():
    folium.CircleMarker(
        location=[fila['Latitud'], fila['Longitud']],
        radius=1.25,                # Más pequeño que antes
        color='#28A745',
        fill=True,
        fill_color='#28A745',
        fill_opacity=1
    ).add_to(mapa)

folium.LayerControl().add_to(mapa)

# Mostrar el mapa
mapa

